In [1]:
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
import sys, importlib, platform
sys.path.append(r"C:\Users\bong2\OneDrive\Python_Codes\00_python_scripts")

import requests
from bs4 import BeautifulSoup
import helper, conn_db
user_agent = helper.user_agent

import time 
from glob import glob
import os

In [80]:
importlib.reload(stock_data_updater)

<module 'stock_data_updater' from 'C:\\Users\\bong2\\OneDrive\\Python_Codes\\00_python_scripts\\stock_data_updater.py'>

In [167]:
def _get_market_index_type(update_date, index_type):
    url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
    header_info = {'User-Agent': user_agent}
    params ={'bld':'dbms/MDC/STAT/standard/MDCSTAT00101',
             'idxIndMidclssCd':f'{index_type}', 
             'share': 1,
            'money': 1,
             'trdDd': str(update_date)}
    r = requests.post(url, data=params, headers=header_info)
    df = pd.DataFrame(r.json()['output'])
    
    col_names = {'IDX_NM': '지수명',
                 'CLSPRC_IDX': '종가',   
                 'OPNPRC_IDX': '시가',
                 'HGPRC_IDX': '고가',
                 'LWPRC_IDX': '저가',
                 'ACC_TRDVOL': '거래량',
                 'ACC_TRDVAL': '거래대금',
                 'MKTCAP': '시가총액'} 
    df.rename(columns=col_names, inplace=True)
    
    num_cols = ['종가', '시가', '고가', '저가', 
                '거래량', '거래대금', '시가총액']
    for col in num_cols:
        df[col] = df[col].str.replace(',','')
        df[col] = df[col].str.replace('-','')
        df[col] = pd.to_numeric(df[col]) 
        
    cols = list(col_names.values())
    return df[cols]

def _get_market_index_types(update_date):
    index_types = {'01':'KRX',
                   '02':'KOSPI',
                   '03':'KOSDAQ',
                   '04':'테마'}
    df = pd.DataFrame()
    for index_type in index_types.keys():
        temp = get_market_index_type(update_date, index_type)
        temp['계열구분'] = index_types[index_type] 
        df = df.append(temp, ignore_index=True)
    
    
    cols = ['지수명','종가', '시가', '고가', '저가', 
            '거래량', '거래대금', '시가총액']
    df['날짜'] = str(update_date)
    df['날짜'] = pd.to_datetime(df['날짜']) 
    cols = ['날짜','계열구분'] + cols  
    return df[cols]

def update_industry_index():
    new_dates = conn_db.from_('코스피_코스닥지수', '최근날짜')[['Date']]
    new_dates = new_dates['Date'].str.replace('-', '').tolist()
    path = conn_db.get_path('업종별지수_raw')
    
    for new_date in new_dates:
        df = get_market_index_types(new_date)
        df.to_pickle(path + f'업종별지수_{new_date}.pkl')
        time.sleep(5)
    
    df = pd.concat([pd.read_pickle(file) for file in glob(path+'*.pkl')], ignore_index=True)
    df.to_pickle(conn_db.get_path('업종별지수_취합본')+'.pkl')
    print('업종별 지수 업데이트 완료')

In [39]:
path = conn_db.get_path('업종별지수_PERPBR배당수익률_raw')
df = helper.concat_pickle(path)
file = conn_db.get_path('업종별지수_PERPBR배당수익률_취합본')+'.pkl'
old_df = pd.read_pickle(file)
# cols = ['지수명','날짜']
# df = helper.add_df(df, old_df, cols).sort_values(by=cols)

In [41]:
cols = ['지수명','날짜']
df = helper.add_df(df, old_df, cols).sort_values(by=cols)

In [42]:
conn_db.to_(df, 'DB_업종별지수', '업종별지수_per')
df.to_pickle(file)

In [33]:
new_dates = conn_db.from_('코스피_코스닥지수', '최근날짜')[['Date']]
new_dates = new_dates['Date'].str.replace('-', '').tolist()
old_dates = df['날짜'].astype(str).str.replace('-','')
new_dates = list(set(new_dates)-set(old_dates))

In [43]:
helper.remove_files(path)

In [48]:
path = conn_db.get_path('업종별지수_PERPBR배당수익률_raw')
df = helper.concat_pickle(path)

In [51]:
p = r"C:\Users\bong2\OneDrive\DataArchive\DB_주식관련\05_업종별지수_PERPBR배당수익률\01_최근\업종별지수_PERPBR배당수익률20210701.pkl"
d = pd.read_pickle(p)
d

,날짜,계열구분,지수명,종가,시가,고가,저가,거래량,거래대금,시가총액
0,2021-07-01,KRX,KRX 300,1977.92,1987.76,1989.05,1975.14,231720173.0,1.397610e+13,2.163614e+15
1,2021-07-01,KRX,KTOP 30,12033.69,12115.74,12136.78,12021.80,42158324.0,6.222806e+12,1.231737e+15
2,2021-07-01,KRX,KRX 100,6863.88,6906.31,6910.82,6855.04,96454303.0,9.837688e+12,1.777696e+15
3,2021-07-01,KRX,KRX Mid 200,1917.31,1912.90,1918.56,1901.87,137654072.0,2.093728e+12,1.085253e+14
4,2021-07-01,KRX,KRX 자동차,2539.99,2531.93,2543.07,2502.03,11712506.0,8.254149e+11,1.476344e+14
...,...,...,...,...,...,...,...,...,...,...
143,2021-07-01,테마,KRX BBIG K-뉴딜지수,3663.57,3653.08,3699.21,3652.83,15852742.0,3.830491e+12,4.139888e+14
144,2021-07-01,테마,KRX 2차전지 K-뉴딜지수,6005.75,6101.60,6177.20,5994.50,8113991.0,2.456619e+12,1.667666e+14
145,2021-07-01,테마,KRX 바이오 K-뉴딜지수,3228.27,3174.01,3241.45,3158.20,11382743.0,1.207968e+12,1.504103e+14
146,2021-07-01,테마,KRX 인터넷 K-뉴딜지수,3171.37,3170.17,3222.92,3164.51,8147473.0,1.139921e+12,1.492907e+14


In [63]:
update_date = '20201207'
index_type = '01'
url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
header_info = {'User-Agent': user_agent}
params ={'bld':'dbms/MDC/STAT/standard/MDCSTAT00701',
         'searchType':'A',
         'idxIndMidclssCd':f'{index_type}',  
         'trdDd': str(update_date)}
r = requests.post(url, data=params, headers=header_info)
df = pd.DataFrame(r.json()['output'])

In [64]:
df.head(2)

,IDX_NM,CLSPRC_IDX,FLUC_TP_CD,PRV_DD_CMPR,FLUC_RT,WT_PER,WT_STKPRC_NETASST_RTO,DIV_YD
0,KRX 300,"1,709.73",1,14.41,0.85,29.54,1.26,1.54
1,KTOP 30,"10,032.85",1,19.71,0.20,30.07,1.33,1.52


In [66]:
col_names = {'IDX_NM' : '지수명',
             'CLSPRC_IDX' :'종가', 
             'WT_PER' :'PER',
             'WT_STKPRC_NETASST_RTO':'PBR',
             'DIV_YD':'배당수익률'}
    
df.rename(columns=col_names, inplace=True)

In [68]:
num_cols = ['종가', 'PER', 'PBR', '배당수익률']
for col in num_cols:
    df[col] = df[col].str.replace(',', '')
    df[col] = df[col].str.replace('-', '')
    df[col] = pd.to_numeric(df[col])

['지수명', '종가', 'PER', 'PBR', '배당수익률']

In [73]:
df[list(col_names.values())]

,지수명,종가,PER,PBR,배당수익률
0,KRX 300,1709.73,29.54,1.26,1.54
1,KTOP 30,10032.85,30.07,1.33,1.52
2,KRX 100,5968.37,29.83,1.26,1.51
3,KRX Mid 200,1581.88,75.13,1.76,0.70
4,KRX 자동차,1920.02,25.30,0.80,1.75
5,KRX 반도체,3096.89,28.20,2.42,0.63
6,KRX 헬스케어,5685.12,209.32,8.23,0.10
7,KRX 은행,645.51,5.09,0.36,5.40
8,KRX 에너지화학,3199.57,NaN,1.24,1.13
9,KRX 철강,1402.73,843.66,0.46,2.15


In [76]:
def get_market_index_type_per(update_date, index_type):
    url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
    header_info = {'User-Agent': user_agent}
    params ={'bld':'dbms/MDC/STAT/standard/MDCSTAT00701',
             'searchType':'A',
             'idxIndMidclssCd':f'{index_type}',  
             'trdDd': str(update_date)}
    r = requests.post(url, data=params, headers=header_info)
    df = pd.DataFrame(r.json()['output'])
    
    col_names = {'IDX_NM' : '지수명',
                 'CLSPRC_IDX' :'종가', 
                 'WT_PER' :'PER',
                 'WT_STKPRC_NETASST_RTO':'PBR',
                 'DIV_YD':'배당수익률'}
    
    df.rename(columns=col_names, inplace=True)
    
    num_cols = ['종가', 'PER', 'PBR', '배당수익률']
    for col in num_cols:
        df[col] = df[col].str.replace(',','')
        df[col] = df[col].str.replace('-','')
        df[col] = pd.to_numeric(df[col]) 
        
    cols = list(col_names.values())
    return df[cols]

def get_market_index_types_per(update_date):
    index_types = {'01':'KRX',
                   '02':'KOSPI',
                   '03':'KOSDAQ',
                   '04':'테마'}
    df = pd.DataFrame()
    for index_type in index_types.keys():
        temp = get_market_index_type(update_date, index_type)
        temp['계열구분'] = index_types[index_type] 
        df = df.append(temp, ignore_index=True)
    
    
    cols = ['지수명','종가', 'PER', 'PBR', '배당수익률']
    df['날짜'] = str(update_date)
    df['날짜'] = pd.to_datetime(df['날짜']) 
    cols = ['날짜','계열구분'] + cols 
    
    # 저장
    return df[cols]

def update_market_index_per():
    new_dates = conn_db.from_('코스피_코스닥지수', '최근날짜')[['Date']]
    new_dates = new_dates['Date'].str.replace('-', '').tolist()
    path = conn_db.get_path('업종별지수_PERPBR배당수익률_raw')
    
    for new_date in new_dates:
        df = get_market_index_types(new_date)
        df.to_pickle(path + f'업종별지수_PERPBR배당수익률{new_date}.pkl')
        time.sleep(5)
    
    df = pd.concat([pd.read_pickle(file) for file in glob(path+'*.pkl')], ignore_index=True)
    df.to_pickle(conn_db.get_path('업종별지수_PERPBR배당수익률_취합본')+'.pkl')
    print('업종별 지수 PER,PBR,배당수익률 업데이트 완료')

In [171]:
update_market_index_per()

업종별 지수 PER,PBR,배당수익률 업데이트 완료
